In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

import os
import sys
import warnings

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))

import matplotlib.pyplot as plt
import matplotlib.colors

import numpy as np
import seaborn as sns

import gan_regression

import importlib
importlib.reload(gan_regression)

plt.rcdefaults()
warnings.filterwarnings("ignore")

%matplotlib qt

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
VERSION = "v4"
FIELD_MODEL = "T89"
MODEL_TYPE = "LOWER_BAND"
pdata_folder = os.path.abspath(r"./../processed_data/chorus_neural_network/")
STAGE_4_folder = os.path.join(pdata_folder, "STAGE_4", VERSION)
CONJUNCTIONS_REFS = np.load(
    file=os.path.join(STAGE_4_folder, f"MODEL_READY_DATA_{VERSION}_{FIELD_MODEL}_{MODEL_TYPE}.npz")
)

X_train = CONJUNCTIONS_REFS["FEATURES"]
y_train = CONJUNCTIONS_REFS["LABELS"].flatten()
day_train = CONJUNCTIONS_REFS["TRAINING_DAY_IDS"].flatten()

MLT_train = CONJUNCTIONS_REFS["TRAINING_MLT"]
MEAN_L = CONJUNCTIONS_REFS["MEAN_L"]
STD_L = CONJUNCTIONS_REFS["STD_L"]


X_valid = CONJUNCTIONS_REFS["VALIDATION_FEATURES"]
y_valid = CONJUNCTIONS_REFS["VALIDATION_LABELS"].flatten()

CONJUNCTIONS_REFS.close()

In [3]:
print(f"Training set shape: {X_train.shape, y_train.shape}")
print(f"Validation set shape: {X_valid.shape, y_valid.shape}")

ax0 = sns.displot(y_train, log_scale=True)
ax0.set(ylabel='N', xlabel='Chorus (pT)', title='Training')

Training set shape: ((156371, 9), (156371,))
Validation set shape: ((1971, 9), (1971,))


In [10]:
# Initialize and train model
model = gan_regression.GANRegression(input_dim=9,
                                     hidden_dim=128,
                                     g_lr=2e-7,
                                     d_lr=2e-7,
                                     epochs_per_output=1)

avg_d_losses, avg_numeric_losses, avg_g_losses = model.train(X_train, y_train, epochs=10, batch_size=32)

using device : cuda
Epoch [1 / 10] 
D Loss: (37.7081 +/- 6.9325) N Loss: (7062.7891 +/- 18333.0603) G Loss: (43.4082 +/- 30.4008)

Epoch [2 / 10] 
D Loss: (24.3999 +/- 4.5638) N Loss: (2183.8060 +/- 7120.8694) G Loss: (4.4477 +/- 3.2772)

Epoch [3 / 10] 
D Loss: (17.9219 +/- 4.0510) N Loss: (38.2605 +/- 48.3139) G Loss: (2.0101 +/- 1.7836)

Epoch [4 / 10] 
D Loss: (16.0105 +/- 3.9297) N Loss: (22.5869 +/- 30.0494) G Loss: (1.7415 +/- 1.6334)

Epoch [5 / 10] 
D Loss: (15.1168 +/- 3.7299) N Loss: (13.9779 +/- 18.0456) G Loss: (1.4661 +/- 1.4027)

Epoch [6 / 10] 
D Loss: (14.5358 +/- 3.7645) N Loss: (11.8787 +/- 12.6067) G Loss: (1.3416 +/- 1.2657)

Epoch [7 / 10] 
D Loss: (14.1097 +/- 3.6865) N Loss: (10.6031 +/- 10.2517) G Loss: (1.2439 +/- 1.1952)

Epoch [8 / 10] 
D Loss: (13.6662 +/- 3.6536) N Loss: (9.6813 +/- 7.2310) G Loss: (1.1052 +/- 1.0139)

Epoch [9 / 10] 
D Loss: (13.2613 +/- 3.6468) N Loss: (9.1589 +/- 6.7534) G Loss: (0.9705 +/- 0.8826)

Epoch [10 / 10] 
D Loss: (12.9368 +/-

In [9]:
threshold = 0.75

y_train_g = model.GENERATE(X_train)

y_train_d = model.DISCRIM(X_train, y_train_g)

passes_discriminator_test = y_train_d >= threshold
plt.scatter(x=y_train[passes_discriminator_test], y=y_train_g[passes_discriminator_test], color="black", s=1.0)

plt.xscale("log")
plt.yscale("log")
plt.xlim(1e-1, 1e3)
plt.ylim(1e-1, 1e3)

(0.1, 1000.0)

In [7]:
y_valid_g = model.GENERATE(X_valid)
y_valid_d = model.DISCRIM(X_valid, y_valid_g)

passes_discriminator_test = y_valid_d >= threshold
plt.scatter(x=y_valid[passes_discriminator_test], y=y_valid_g[passes_discriminator_test], color="red", s=1.0)
plt.xscale("log")
plt.yscale("log")
plt.xlim(1e-1, 1e3)


plt.ylim(1e-1, 1e3)

(0.1, 1000.0)

In [ ]:
print(y_train_g)
print(np.log10(y_train_g))

In [11]:
not_negative = y_train_g > 0
passes_discriminator_test = y_train_d >= threshold
passes_both_filters = not_negative & passes_discriminator_test

a, b = np.polyfit(np.log10(y_train[passes_both_filters]), np.log10(y_train_g[passes_both_filters]), 1)

x_fit = np.logspace(-1, 4, 1000)

# Create the line of best fit
y_fit = a * np.log10(x_fit) + b

# Plot the line of best fit
plt.plot(x_fit, 10**y_fit, color='red')
plt.plot(x_fit, x_fit, color='green')


print(a)


0.11282530245189473


In [ ]:
not_negative = y_valid_g > 0
passes_discriminator_test = y_valid_d >= threshold
passes_both_filters = not_negative & passes_discriminator_test

a, b = np.polyfit(np.log10(y_valid[passes_both_filters]), np.log10(y_valid_g[passes_both_filters]), 1)

x_fit = np.logspace(-1, 4, 1000)

# Create the line of best fit
y_fit = a * np.log10(x_fit) + b

# Plot the line of best fit
plt.plot(x_fit, 10**y_fit, color='red')
plt.plot(x_fit, x_fit, color='green')


print(a)